In [1]:
pip install tensorflow

In [2]:
pip install optree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.4 MB/s eta 0:00:00


In [3]:
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

import tensorflow as tf; tf.keras


from tensorflow.keras.layers import Dense, Input,Dropout, Flatten, Activation,Convolution2D, MaxPooling2D
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid


#from tensorflow.python.keras.models import Sequential
#from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
#from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
K.clear_session()



data_entrenamiento = '/content/drive/MyDrive/Machine_Learning/Datasets/DatasetPajaros/train'
data_validacion = '/content/drive/MyDrive/Machine_Learning/Datasets/DatasetPajaros/test'

In [10]:
epocas=2000
longitud, altura = 150, 150
batch_size = 32
pasos = 1
validation_steps = 300
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 3
lr = 0.0004

In [11]:
##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 438 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [12]:
cnn = Sequential()
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

cnn.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [13]:
cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Epoch 1/2000
1/1 [==============================] - ETA: 0s - loss: 1.1070 - accuracy: 0.4062

1/1 [==============================] - 20s 20s/step - loss: 1.1070 - accuracy: 0.4062 - val_loss: 6.3217 - val_accuracy: 0.5333
Epoch 2/2000
1/1 [==============================] - 5s 5s/step - loss: 13.0855 - accuracy: 0.4062
Epoch 3/2000
1/1 [==============================] - 3s 3s/step - loss: 10.3593 - accuracy: 0.3750
Epoch 4/2000
1/1 [==============================] - 3s 3s/step - loss: 14.9175 - accuracy: 0.3750
Epoch 5/2000
1/1 [==============================] - 0s 461ms/step - loss: 4.3557 - accuracy: 0.5938
Epoch 6/2000
1/1 [==============================] - 1s 1s/step - loss: 2.3743 - accuracy: 0.5000
Epoch 7/2000
1/1 [==============================] - 4s 4s/step - loss: 2.3494 - accuracy: 0.4688
Epoch 8/2000
1/1 [==============================] - 2s 2s/step - loss: 3.5306 - accuracy: 0.5000
Epoch 9/2000
1/1 [==============================] - 1s 983ms/step - loss: 0.9413 - accuracy: 0.6818
Epoch 10/2000
1/1 [==============================] - 2s 2s/step - loss: 0.5363 - accura

In [14]:
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
def predict(file):

  x = load_img(file, target_size=(longitud, altura))

  x = img_to_array(x)

  x = np.expand_dims(x, axis=0)

  array = cnn.predict(x)
  print(array)
  result = array[0]
  print(result)
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Cuckoo")
  elif answer == 1:
    print("pred: Cockato")
  elif answer == 2:
    print("pred: Golden Weaver")

  return answer

In [18]:
#EJEMPLO DE COCKATO
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model


file='/content/drive/MyDrive/Machine_Learning/Datasets/DatasetPajaros/test/BLACK COCKATO/1.jpg'
predict(file)
x = load_img(file, target_size=(longitud, altura))
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
array = cnn.predict(x)

1/1 [==============================] - 0s 26ms/step
[[0. 1. 0.]]
[0. 1. 0.]
pred: Cockato
1/1 [==============================] - 0s 24ms/step


In [19]:
#EJEMPLO DE AFRICAN EMERALD CUCKOO
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model


file='/content/drive/MyDrive/Machine_Learning/Datasets/DatasetPajaros/test/AFRICAN EMERALD CUCKOO/1.jpg'
predict(file)
x = load_img(file, target_size=(longitud, altura))
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
array = cnn.predict(x)

1/1 [==============================] - 0s 26ms/step
[[1. 0. 0.]]
[1. 0. 0.]
pred: Cuckoo
1/1 [==============================] - 0s 30ms/step


In [20]:
#EJEMPLO DE AFRICAN EASTERN GOLD WEAVER
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model


file='/content/drive/MyDrive/Machine_Learning/Datasets/DatasetPajaros/test/EASTERN GOLDEN WEAVER/1.jpg'
predict(file)
x = load_img(file, target_size=(longitud, altura))
x = img_to_array(x)
x = np.expand_dims(x, axis=0)
array = cnn.predict(x)

1/1 [==============================] - 0s 17ms/step
[[0. 0. 1.]]
[0. 0. 1.]
pred: Golden Weaver
1/1 [==============================] - 0s 17ms/step


In [ ]:
target_dir = './modelo/'

longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn1 = load_model(modelo)
cnn1.load_weights(pesos_modelo)